# Coupled Cluster Calculations of Infinite Nuclear Matter in the Complete Basis Limit Using Bayesian Machine Learning
## Source Code

Date Created: August 17, 2024

Last Modified: August 17, 2024

## Imports

In [ ]:
%%time
##############################
##          IMPORTS         ##
##############################
# THIRD PARTY IMPORTS
# For arrays
import numpy as np
# For importing the data set
import pandas as pd
# For graphing
import matplotlib.pyplot as plt
# For calculating the r2 score
from sklearn.metrics import r2_score

##############################
##        DIRECTORIES       ##
##############################
# Directories to find data files and save images
data_dir = 'raw_data/
images_dir = 'img/'

## Figure 1

### Left

### Right

## Figure 2

### Left

### Right

## Figure 3